# サンプル 01: OpenAI SDK を使ったクイックチャット

このノートブックでは、Microsoft Foundry Local を使用して OpenAI SDK を活用し、クイックチャットを行う方法を示します。

## 概要

この例では以下を示します：
- Foundry Local を使用した OpenAI Python SDK の利用方法
- Azure OpenAI とローカル Foundry 設定の両方への対応
- 適切なエラーハンドリングとフォールバック戦略の実装
- FoundryLocalManager を使用したサービス管理


## 前提条件

以下を確認してください:
1. Foundry Localがインストールされ、実行中であること
2. モデルがロードされていること（例: `foundry model run phi-4-mini`）
3. 必要なPythonパッケージがインストールされていること

### 依存関係のインストール


In [ ]:
# Install required packages if not already installed
!pip install openai foundry-local-sdk

## 必要なライブラリをインポート


In [ ]:
import os
import sys
from openai import OpenAI

try:
    from foundry_local import FoundryLocalManager
    FOUNDRY_SDK_AVAILABLE = True
    print("✅ Foundry Local SDK is available")
except ImportError:
    FOUNDRY_SDK_AVAILABLE = False
    print("⚠️ Foundry Local SDK not available, will use manual configuration")

## 設定

設定を行います。以下のいずれかを使用できます：
1. **Azure OpenAI**（クラウドベース）
2. **Foundry Local**（ローカル推論）

### オプション 1: Azure OpenAI の設定


In [ ]:
# Azure OpenAI Configuration (uncomment and set your values)
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://your-resource.openai.azure.com"
# os.environ["AZURE_OPENAI_API_KEY"] = "your-api-key"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-08-01-preview"
# os.environ["MODEL"] = "your-deployment-name"

### オプション2: ファウンドリのローカル構成


In [ ]:
# Foundry Local Configuration
MODEL_ALIAS = "phi-4-mini"  # Change this to your preferred model
BASE_URL = "http://localhost:8000"  # Default Foundry Local URL
API_KEY = ""  # Usually not needed for local

## クライアントの初期化

設定に基づいてOpenAIクライアントを作成します:


In [ ]:
def create_client():
    """Create and configure OpenAI client."""
    
    # Check for Azure OpenAI configuration
    azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")
    azure_api_key = os.environ.get("AZURE_OPENAI_API_KEY")
    azure_api_version = os.environ.get("AZURE_OPENAI_API_VERSION", "2024-08-01-preview")
    
    if azure_endpoint and azure_api_key:
        # Azure OpenAI path
        model = os.environ.get("MODEL", "your-deployment-name")
        client = OpenAI(
            base_url=f"{azure_endpoint}/openai",
            api_key=azure_api_key,
            default_query={"api-version": azure_api_version},
        )
        print(f"🌐 Using Azure OpenAI with model: {model}")
        return client, model
    
    # Foundry Local path
    if FOUNDRY_SDK_AVAILABLE:
        try:
            # Use FoundryLocalManager for proper service management
            manager = FoundryLocalManager(MODEL_ALIAS)
            model_info = manager.get_model_info(MODEL_ALIAS)
            
            # Configure OpenAI client to use local Foundry service
            client = OpenAI(
                base_url=manager.endpoint,
                api_key=manager.api_key  # API key is not required for local usage
            )
            model = model_info.id
            print(f"🏠 Using Foundry Local SDK with model: {model}")
            return client, model
        except Exception as e:
            print(f"⚠️ Could not use Foundry SDK ({e}), falling back to manual configuration")
    
    # Fallback to manual configuration
    client = OpenAI(
        base_url=f"{BASE_URL}/v1",
        api_key=API_KEY
    )
    model = MODEL_ALIAS
    print(f"🔧 Using manual configuration with model: {model}")
    return client, model

# Initialize the client
client, model = create_client()

## 基本的なチャット例

簡単なチャットのやり取りを試してみましょう:


In [ ]:
def chat_with_model(prompt, max_tokens=128):
    """Send a chat message to the model and get a response."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens
        )
        
        return response.choices[0].message.content
    except Exception as e:
        return f"Error: {e}"

# Test with a simple prompt
prompt = "Say hello from Foundry Local and explain what you are in one sentence."
print(f"👤 User: {prompt}")
print("\n🤖 Assistant:")
response = chat_with_model(prompt)
print(response)

## インタラクティブチャットセッション

さまざまな種類のプロンプトを試して、モデルの反応を確認してください:


In [ ]:
# Example prompts to try
example_prompts = [
    "What are the benefits of edge AI?",
    "Explain the difference between local and cloud AI inference.",
    "Write a short Python function to calculate the factorial of a number.",
    "What is Microsoft Foundry Local?"
]

for i, prompt in enumerate(example_prompts, 1):
    print(f"\n{'='*50}")
    print(f"Example {i}: {prompt}")
    print(f"{'='*50}")
    
    response = chat_with_model(prompt, max_tokens=200)
    print(response)

## 高度な使用法: ストリーミングレスポンス

より長いレスポンスの場合、ストリーミングを使用することで、より良いユーザー体験を提供できます:


In [ ]:
def chat_with_streaming(prompt, max_tokens=300):
    """Send a chat message with streaming response."""
    try:
        stream = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=max_tokens,
            stream=True
        )
        
        print("🤖 Assistant (streaming):")
        full_response = ""
        for chunk in stream:
            if chunk.choices[0].delta.content is not None:
                content = chunk.choices[0].delta.content
                print(content, end="", flush=True)
                full_response += content
        print("\n")  # New line after streaming
        return full_response
    except Exception as e:
        return f"Error: {e}"

# Test streaming with a longer prompt
streaming_prompt = "Explain the advantages of running AI models locally on device versus in the cloud. Include privacy, latency, and cost considerations."
print(f"👤 User: {streaming_prompt}\n")
streaming_response = chat_with_streaming(streaming_prompt)

## エラー処理と診断

サービスの状態と利用可能なモデルを確認しましょう:


In [ ]:
def check_service_health():
    """Check Foundry Local service health."""
    try:
        # Try to list available models
        models_response = client.models.list()
        models = [model.id for model in models_response.data]
        
        print("✅ Service is healthy")
        print(f"📋 Available models: {models}")
        print(f"🎯 Current model: {model}")
        
        if model in models:
            print("✅ Current model is available")
        else:
            print("⚠️ Current model may not be loaded")
            
    except Exception as e:
        print(f"❌ Service check failed: {e}")
        print("\n🔧 Troubleshooting tips:")
        print("1. Make sure Foundry Local is running")
        print("2. Check if a model is loaded: foundry service ps")
        print("3. Verify the endpoint URL is correct")

check_service_health()

## カスタムプロンプトのテスト

以下のセルを使用して、自分のプロンプトをテストしてください:


In [ ]:
# Enter your custom prompt here
custom_prompt = "Write a haiku about artificial intelligence running on edge devices."

print(f"👤 User: {custom_prompt}\n")
custom_response = chat_with_model(custom_prompt, max_tokens=100)
print(f"🤖 Assistant: {custom_response}")

## 概要

このノートブックでは以下を示しました：

1. **✅ クライアント設定**: Foundry Localを使用したOpenAI SDKの設定方法
2. **✅ 基本的なチャット**: シンプルなリクエストとレスポンスのやり取り
3. **✅ ストリーミング**: リアルタイムのレスポンス生成
4. **✅ エラーハンドリング**: 頑健なエラーハンドリングと診断
5. **✅ サービスの状態確認**: モデルの利用可能性とサービスの状態確認

### 次のステップ

- **サンプル 02**: Azure OpenAI対応の高度なSDK統合
- **サンプル 04**: Chainlitチャットアプリケーションの構築
- **サンプル 05**: マルチエージェントオーケストレーションシステム
- **サンプル 06**: インテリジェントなモデルルーティング

### Foundry Localの主な利点

- 🔒 **プライバシー**: データがデバイス外に出ることはありません
- ⚡ **スピード**: 低遅延のローカル推論
- 💰 **コスト**: API使用料が不要
- 🔌 **オフライン**: インターネット接続なしで動作
- 🛠️ **互換性**: OpenAI互換API
